<a href="https://colab.research.google.com/github/Taiga10969/Lecture-Transformer/blob/main/copy_code_Transformer_05_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 写経して理解するTransformer_05：Transformer

01〜04で作成してきた各機構を用いてTransformerを構築していく．

## 必要ライブラリのインポート

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

## 各クラスの定義

In [2]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model) # torch.nn の nn.Embedding を利用
        self.d_model = d_model

    def forward(self, x):
        # x：torch.Size([1, 7]) << [batch_size, src_len]
        embedded = self.embedding(x)
        # embedded : torch.Size([1, 7, 128]) << [batch_size, src_len, d_model]
        return embedded

In [3]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len):
        super().__init__()

        self.d_model = d_model

        pe = torch.zeros(max_seq_len, d_model)

        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2*i) / d_model)))
                pe[pos, i+1] = math.cos(pos / (10000 ** ((2*i) / d_model)))

        self.pe = nn.Parameter(pe, requires_grad=False)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        ret = math.sqrt(self.d_model) * x + self.pe[:seq_len, :].unsqueeze(0)
        return ret

    def get_pe(self, x):
        batch_size, seq_len, _ = x.size()
        return self.pe[:seq_len, :].unsqueeze(0)

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)

        self.W_o = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        query = self.W_q(query)
        key = self.W_k(key)
        value = self.W_v(value)

        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        attention_score = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.depth, dtype=torch.float32))

        if mask is not None:
            attention_score += mask

        attention_weights = F.softmax(attention_score, dim=-1)
        attention = torch.matmul(attention_weights, value)
        attention = attention.permute(0, 2, 1, 3).contiguous()
        attention = attention.view(batch_size, -1, self.d_model)

        return self.W_o(attention)

In [5]:
class FeedForwardNetwork(nn.Module):

  def __init__(self, dim, hidden_dim, dropout=0.1):
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.linear_1 = nn.Linear(dim, hidden_dim)
    self.relu = nn.ReLU()
    self.linear_2 = nn.Linear(hidden_dim, dim)

  def forward(self, x):
    x = self.linear_1(x)
    x = self.relu(x)
    x = self.dropout(x)
    x = self.linear_2(x)
    return x

## Transformer Encoder の構築
Encoderブロックを複数積層してEncoderを構築するため，<br>
EncoderBlockをクラスで定義し，そのクラスをインスタンスとして複数呼び出すEncoderクラスを作成する．

In [6]:
class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, hidden_dim, dropout=0.1):
        super().__init__()

        self.MHA = MultiHeadAttention(d_model, num_heads)
        self.layer_norm1 = nn.LayerNorm([d_model])
        self.layer_norm2 = nn.LayerNorm([d_model])
        self.FFN = FeedForwardNetwork(d_model, hidden_dim)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
      # self-Attention >> Q，K，Vに同じデータを流す
      Q = K = V = _x = x

      x = self.MHA(Q, K, V, mask)
      x = self.dropout_1(x)
      x = x + _x # Add
      x = self.layer_norm1(x) # Norm

      _x = x

      x = self.FFN(x)
      x = self.dropout_2(x)
      x = x + _x #Add
      x = self.layer_norm2(x) # Norm

      return x

In [7]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, max_seq_len, hidden_dim, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.embed = Embedder(vocab_size, d_model)
        self.PE = PositionalEncoder(d_model, max_seq_len)
        self.dropout = nn.Dropout(dropout)
        self.EncoderBlocks = nn.ModuleList([EncoderBlock(d_model, num_heads, hidden_dim) for _ in range(6)])

    def forward(self, x, mask=None):
        x = self.embed(x)
        x = x*(self.d_model**0.5)
        x = self.PE(x)
        x = self.dropout(x)

        if mask is not None:
            mask = torch.where(mask == 1, torch.tensor(0), torch.tensor(float('-inf')))

        for i in range(6):
            x = self.EncoderBlocks[i](x, mask)
        return x

## Transformer Decoder の構築
Decoderブロックを複数積層してDecoderを構築するため，<br>
DecoderBlockをクラスで定義し，そのクラスをインスタンスとして複数呼び出すDecoderクラスを作成する．

In [8]:
class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, hidden_dim, dropout=0.1):
        super().__init__()

        self.MHA_1 = MultiHeadAttention(d_model, num_heads)
        self.MHA_2 = MultiHeadAttention(d_model, num_heads)

        self.layer_norm_1 = nn.LayerNorm([d_model])
        self.layer_norm_2 = nn.LayerNorm([d_model])
        self.layer_norm_3 = nn.LayerNorm([d_model])

        self.FFN = FeedForwardNetwork(d_model, hidden_dim)

        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)

    def forward(self, tgt, encoder_output, decoder_mask=None, encoder_mask=None):
        # Decoder の一つ目のMHAは，self AttentionのMasked Multi-Head Attentionとなっており，入力QKVは，全て同じデータ
        Q = K = V = _x = tgt

        # 情報の先読みを防止するマスクの作成
        seq_len = tgt.size(1)
        mask = torch.triu(torch.ones(seq_len, seq_len) * -float('inf'), diagonal=1)
        mask = mask.unsqueeze(0).unsqueeze(1) # バッチとヘッドの次元を追加

        if decoder_mask is not None:
            decoder_mask = torch.where(decoder_mask == 1, torch.tensor(0), torch.tensor(float('-inf')))
            mask = mask + decoder_mask

        x = self.MHA_1(Q, K, V, mask)
        x = self.dropout_1(x)
        x = x + _x
        x = self.layer_norm_1(x)

        # ここまでアーキテクチャ図のDecoderの下半分
        # ここからアーキテクチャ図のDecoderの上半分

        Q = x # queryには下半分からの出力を
        _x = x
        K = V = encoder_output # key,valueにはencoderからの出力を

        x = self.MHA_2(Q, K, V, encoder_mask)
        x = self.dropout_2(x)
        x = x + _x
        x = self.layer_norm_2(x)

        _x = x

        x = self.FFN(x)
        x = self.dropout_3(x)
        x = x + _x
        x = self.layer_norm_3(x)

        return x

In [9]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, max_seq_len, hidden_dim, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.embed = Embedder(vocab_size, d_model)

        self.PE = PositionalEncoder(d_model, max_seq_len)

        self.DecoderBlocks = nn.ModuleList([DecoderBlock(d_model, num_heads, hidden_dim) for _ in range(6)])
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, tgt, encoder_output, mask=None, encoder_mask=None):
        x = self.embed(tgt)
        x = x*(self.d_model**0.5)
        x = self.PE(x)
        x = self.dropout(x)

        for i in range(6):
            x = self.DecoderBlocks[i](x, encoder_output, mask, encoder_mask)

        x = self.linear(x)
        return x

## Transformer実装

In [10]:
class Transformer(nn.Module):
    def __init__(self, vocab_size_en, vocab_size_jp, d_model, num_heads, max_seq_len, hidden_dim):
        super().__init__()
        self.encoder = Encoder(vocab_size_en, d_model, num_heads, max_seq_len, hidden_dim)
        self.decoder = Decoder(vocab_size_jp, d_model, num_heads, max_seq_len, hidden_dim)

    def forward(self, src, tgt, src_mask, tgt_mask):
        encoder_output = self.encoder(src, src_mask)
        output = self.decoder(tgt, encoder_output, tgt_mask, src_mask)
        return output

## Transformerの動作確認

In [11]:
from transformers import GPT2Tokenizer, AutoTokenizer
# 文字列をトークンのID列に変換するトークナイザーはGPT-2のものを使用します
# 日本語用と英語用でそれぞれ定義
tokenizer_jp = AutoTokenizer.from_pretrained("colorfulscoop/gpt2-small-ja",
                                               add_bos_token = True
                                               )
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',
                                            add_bos_token = True
                                            )

tokenizer_jp.add_special_tokens({'bos_token': '<sos>'})
tokenizer_jp.add_special_tokens({'pad_token': '<pad>'})
tokenizer_jp.add_special_tokens({'eos_token': '<eos>'})
tokenizer.add_special_tokens({'bos_token': '<sos>'})
tokenizer.add_special_tokens({'pad_token': '<pad>'})
tokenizer.add_special_tokens({'eos_token': '<eos>'})

# 入力するtextを定義
text_en = 'That is a great idea.'

# 出力を期待するtext (翻訳結果) を定義
text_jp = "それは素晴らしいアイデアだ。"

# 入力するtextをトークナイザーを用いて数値(id)化する
inputs = tokenizer(text_en,
                   padding=True,
                   truncation=True,
                   max_length=64,
                   return_tensors='pt',)

outputs = tokenizer_jp(text_jp,
                        padding=True,
                        truncation=True,
                        max_length=64,
                        return_tensors='pt',)

# モデルに入力するID列を表示・形状を確認

print("inputs['input_ids']　: ", inputs['input_ids'])
print("inputs['input_ids'].shape　: ", inputs['input_ids'].shape)
print("inputs['input_ids'] token : ", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

print("outputs['input_ids']　: ", outputs['input_ids'])
print("outputs['input_ids'].shape　: ", outputs['input_ids'].shape)
print("outputs['input_ids'] token : ", tokenizer_jp.convert_ids_to_tokens(outputs['input_ids'][0]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


inputs['input_ids']　:  tensor([[50257,  2504,   318,   257,  1049,  2126,    13]])
inputs['input_ids'].shape　:  torch.Size([1, 7])
inputs['input_ids'] token :  ['<sos>', 'That', 'Ġis', 'Ġa', 'Ġgreat', 'Ġidea', '.']
outputs['input_ids']　:  tensor([[ 1901, 13559,  9994,   328,     7]])
outputs['input_ids'].shape　:  torch.Size([1, 5])
outputs['input_ids'] token :  ['それは', '素晴らしい', 'アイデア', 'だ', '。']


In [12]:
vocab_size_jp = tokenizer_jp.vocab_size+3
vocab_size_en = tokenizer.vocab_size+3
d_model = 128
num_heads = 4
max_seq_len = 256
hidden_dim = 256

model = Transformer(vocab_size_en = vocab_size_en,
                    vocab_size_jp = vocab_size_jp,
                    d_model = d_model,
                    num_heads = num_heads,
                    max_seq_len = max_seq_len,
                    hidden_dim = hidden_dim)

output = model(src = inputs['input_ids'],
                 tgt = outputs['input_ids'],
                 src_mask = inputs['attention_mask'],
                 tgt_mask = outputs['attention_mask'])

print("output.shape : ", output.shape)

outputs_IDs = torch.argmax(output, dim=-1)
print("outputs_IDs : ", outputs_IDs)

# モデルが出力した数値(id)を各単語に逆変換
generated_text = tokenizer.decode(outputs_IDs[0])
print("generated_text : ", generated_text)
print("model_output token : ", tokenizer.convert_ids_to_tokens(outputs_IDs[0]))

output.shape :  torch.Size([1, 5, 32003])
outputs_IDs :  tensor([[25180, 18853, 18457,  8298, 19390]])
generated_text :  acketsimity capac001 Request
model_output token :  ['ackets', 'imity', 'Ġcapac', '001', 'ĠRequest']


現状は何も学習していない乱数によるパラメータなので，期待している次単語予測による文章が生成できていないが，<br>
Transformerの大まかなデータの流れはこのようになっている．<br>
<br>
このプログラムを参考に，色々データの中身や形状をPrint()して表示させてみて，理解につなげてみよう！<br>
<br>
以上<br>
<br>
中部大学 機械知覚&ロボティクスグループ（藤吉研究室）<br>
増田 大河
